In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve
from tensorflow.keras.callbacks import EarlyStopping
import warnings
import time
warnings.filterwarnings('ignore')

2024-05-21 00:15:39.144210: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 00:15:39.144303: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 00:15:39.277665: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def create_data_generator(data_dir, batch_size):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        zoom_range=0.1,
        brightness_range=[0.5, 1.5],
        horizontal_flip=True,
        fill_mode='nearest'
    )
    generator = datagen.flow_from_directory(
        directory=data_dir,
        target_size=(224, 224),  # Resize images to 224x224 to match the input size of the model
        batch_size=batch_size,
        class_mode='binary'  # Binary labels
    )
    return generator

# Create generators
batch_size = 128
train_generator = create_data_generator('/kaggle/input/morph-splitted/train', batch_size)
val_generator = create_data_generator('/kaggle/input/morph-splitted/val', batch_size)
test_generator = create_data_generator('/kaggle/input/morph-splitted/test', batch_size)


Found 24000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.


In [4]:
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
     
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid', dtype='float32')  # Ensure the output layer is in float32
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 788544)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 788544)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │       788,545 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 790,593 (3.02 MB)

 Trainable params: 790,465 (3.02 MB)

 Non-trainable params: 128 (512.00 B)

In [5]:
epochs = 3
batch_size = 128
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

Epoch 1/3
  1/188 ━━━━━━━━━━━━━━━━━━━━ 1:47:16 34s/step - accuracy: 0.4297 - loss: 0.9111

I0000 00:00:1716250628.154130     129 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


188/188 ━━━━━━━━━━━━━━━━━━━━ 774s 4s/step - accuracy: 0.8590 - loss: 3.0610 - val_accuracy: 0.6344 - val_loss: 0.7421
Epoch 2/3
188/188 ━━━━━━━━━━━━━━━━━━━━ 582s 3s/step - accuracy: 0.9683 - loss: 0.1003 - val_accuracy: 0.9646 - val_loss: 0.1378
Epoch 3/3
188/188 ━━━━━━━━━━━━━━━━━━━━ 582s 3s/step - accuracy: 0.9732 - loss: 0.0915 - val_accuracy: 0.7769 - val_loss: 0.4844


In [6]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 195s 3s/step - accuracy: 0.7789 - loss: 0.4801
Test accuracy: 0.7758749723434448


In [7]:
from sklearn.metrics import classification_report

# Generate predictions
predictions = model.predict(test_generator)

# Convert predictions to binary labels
predicted_labels = (predictions > 0.5).astype(int)

# Get true labels
true_labels = test_generator.classes

# Generate classification report
report = classification_report(true_labels, predicted_labels)
print(report)


63/63 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step
              precision    recall  f1-score   support

           0       0.63      0.85      0.72      5000
           1       0.39      0.16      0.22      3000

    accuracy                           0.59      8000
   macro avg       0.51      0.51      0.47      8000
weighted avg       0.54      0.59      0.54      8000



In [8]:
fm_generator = create_data_generator('/kaggle/input/mad-benchmark/FaceMorpher', batch_size)
mg1_generator = create_data_generator('/kaggle/input/mad-benchmark/MIPGAN_I', batch_size)
mg2_generator = create_data_generator('/kaggle/input/mad-benchmark/MIPGAN_II', batch_size)
oc_generator = create_data_generator('/kaggle/input/mad-benchmark/OpenCV', batch_size)
wm_generator = create_data_generator('/kaggle/input/mad-benchmark/Webmorph', batch_size)

Found 1204 images belonging to 2 classes.
Found 1204 images belonging to 2 classes.
Found 1203 images belonging to 2 classes.
Found 1188 images belonging to 2 classes.
Found 704 images belonging to 2 classes.


In [10]:
test_loss, test_accuracy = model.evaluate(wm_generator)

6/6 ━━━━━━━━━━━━━━━━━━━━ 25s 4s/step - accuracy: 0.3020 - loss: 3.1712


In [11]:
datasets = [fm_generator, mg1_generator, mg2_generator, oc_generator,wm_generator]
results = []
for i in datasets:
    test_loss, test_accuracy = model.evaluate(i, batch_size=16)
    results.append(test_accuracy)

10/10 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.1814 - loss: 4.8413
10/10 ━━━━━━━━━━━━━━━━━━━━ 79s 8s/step - accuracy: 0.1853 - loss: 2.4774
10/10 ━━━━━━━━━━━━━━━━━━━━ 81s 8s/step - accuracy: 0.1929 - loss: 2.3490
10/10 ━━━━━━━━━━━━━━━━━━━━ 39s 4s/step - accuracy: 0.1716 - loss: 3.4034
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.2912 - loss: 3.3278


In [12]:
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
for i,j in zip(names, results):
    print(i, ": ", j)

FaceMorpher :  0.18189369142055511
MIPGAN_I :  0.1901993304491043
MIPGAN_II :  0.19617623090744019
OpenCV :  0.1734006702899933
Webmorph :  0.28977271914482117


### **apcer_at_fixed_bpcer**

In [13]:
def calculate_apcer_at_fixed_bpcer(fpr, tpr, thresholds, fixed_bpcer):
    """Calculate the APCER at a fixed BPCER."""
    fpr_target = fixed_bpcer
    closest_fpr_index = np.argmin(np.abs(fpr - fpr_target))
    corresponding_apcer = 1 - tpr[closest_fpr_index]
    corresponding_threshold = thresholds[closest_fpr_index]
    return corresponding_apcer, corresponding_threshold

In [14]:
# def calculate_apcer_at_fixed_bpcer(fpr, tpr, thresholds, fixed_bpcer):
#     """Calculate the APCER at a fixed BPCER."""
#     tpr_target = 1 - fixed_bpcer
#     closest_tpr_index = np.argmin(np.abs(tpr - tpr_target))
#     corresponding_apcer = fpr[closest_tpr_index]
#     corresponding_threshold = thresholds[closest_tpr_index]
#     return corresponding_apcer, corresponding_threshold


In [15]:
datasets = [ fm_generator, mg1_generator, mg2_generator, oc_generator,wm_generator]
names = [ "FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
fixed_bpcer_values = [0.01, 0.1, 0.2]  # Define fixed BPCER values
all_results = []

# Iterate over each dataset
for dataset, name in zip(datasets, names):
    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(dataset, steps=dataset.samples // dataset.batch_size)
    
    # Predictions and true labels
    predictions = model.predict(dataset)
    true_labels = dataset.classes
    if predictions.ndim > 1 and predictions.shape[1] > 1:
        predictions = predictions[:, 1]

    # ROC curve metrics
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    
    # Calculate APCER for each BPCER
    for fixed_bpcer in fixed_bpcer_values:
        apcer, threshold = calculate_apcer_at_fixed_bpcer(fpr, tpr, thresholds, fixed_bpcer)
        result = {
            "Dataset": name,
            "Fixed BPCER": f"{fixed_bpcer * 100:.1f}%",
            "APCER": f"{apcer:.3f}",
            "Threshold": f"{threshold:.3f}",
            "Test Accuracy": f"{test_accuracy:.2f}"
        }
        all_results.append(result)

# Create DataFrame
df_results = pd.DataFrame(all_results)

# Display the DataFrame
print(df_results)


9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.1837 - loss: 4.7958
10/10 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 57s 6s/step - accuracy: 0.1880 - loss: 2.4147
10/10 ━━━━━━━━━━━━━━━━━━━━ 59s 6s/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 65s 7s/step - accuracy: 0.1951 - loss: 2.3785
10/10 ━━━━━━━━━━━━━━━━━━━━ 109s 11s/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 82s 10s/step - accuracy: 0.1823 - loss: 3.3998
10/10 ━━━━━━━━━━━━━━━━━━━━ 30s 3s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 35s 8s/step - accuracy: 0.3001 - loss: 3.2040
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step
        Dataset Fixed BPCER  APCER Threshold Test Accuracy
0   FaceMorpher        1.0%  0.996     0.683          0.18
1   FaceMorpher       10.0%  0.900     0.123          0.18
2   FaceMorpher       20.0%  0.789     0.033          0.18
3      MIPGAN_I        1.0%  0.987     0.522          0.18
4      MIPGAN_I       10.0%  0.917     0.265          0.18
5      MIPGAN_I       20.0%  0.820     0.156          0.18
6     MIPGAN_II        1.0%  0.983  

### **bpcer_at_fixed_apcer**

In [16]:
def calculate_bpcer_at_fixed_apcer(fpr, tpr, thresholds, fixed_apcer):
    """Calculate the BPCER at a fixed APCER."""
    closest_index = np.argmin(np.abs(fpr - fixed_apcer))
    corresponding_bpcer = 1 - tpr[closest_index]
    corresponding_threshold = thresholds[closest_index]
    return corresponding_bpcer, corresponding_threshold


In [17]:
# Define datasets, model predictions, and fixed APCER values
datasets = [ fm_generator, mg1_generator, mg2_generator, oc_generator,wm_generator]
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
fixed_apcer_values = [0.01, 0.1, 0.2]
all_results = []

# Iterate over each dataset
for dataset, name in zip(datasets, names):
    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(dataset, steps=dataset.samples // dataset.batch_size)
    
    # Predictions and true labels
    predictions = model.predict(dataset)
    true_labels = dataset.classes
    if predictions.ndim > 1 and predictions.shape[1] > 1:
        predictions = predictions[:, 1]

    # ROC curve metrics
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    
    # Calculate BPCER for each fixed APCER
    for fixed_apcer in fixed_apcer_values:
        bpcer, threshold = calculate_bpcer_at_fixed_apcer(fpr, tpr, thresholds, fixed_apcer)
        result = {
            "Dataset": name,
            "Fixed APCER": f"{fixed_apcer * 100:.1f}%",
            "BPCER": f"{bpcer:.3f}",
            "Threshold": f"{threshold:.3f}",
            "Test Accuracy": f"{test_accuracy:.2f}"
        }
        all_results.append(result)

# Create DataFrame
df_results = pd.DataFrame(all_results)

# Display the DataFrame
print(df_results)


9/9 ━━━━━━━━━━━━━━━━━━━━ 65s 8s/step - accuracy: 0.1957 - loss: 4.6174
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 106s 12s/step - accuracy: 0.1878 - loss: 2.3891
10/10 ━━━━━━━━━━━━━━━━━━━━ 59s 6s/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 56s 6s/step - accuracy: 0.1996 - loss: 2.3193
10/10 ━━━━━━━━━━━━━━━━━━━━ 59s 6s/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 28s 3s/step - accuracy: 0.1682 - loss: 3.4621
10/10 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.2984 - loss: 3.2771
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step
        Dataset Fixed APCER  BPCER Threshold Test Accuracy
0   FaceMorpher        1.0%  0.981     0.481          0.18
1   FaceMorpher       10.0%  0.876     0.127          0.18
2   FaceMorpher       20.0%  0.753     0.037          0.18
3      MIPGAN_I        1.0%  0.982     0.571          0.19
4      MIPGAN_I       10.0%  0.898     0.264          0.19
5      MIPGAN_I       20.0%  0.841     0.178          0.19
6     MIPGAN_II        1.0%  0.996   

In [18]:
# model.save('model.h5')  # Saves the entire model to a single HDF5 file.

In [19]:
import numpy as np
from sklearn.metrics import roc_curve
from tensorflow.keras.backend import clear_session

def calculate_eer(true_labels, predictions):
    """Calculate the Equal Error Rate (EER) and the corresponding threshold."""
    # Compute ROC curve
    fpr, tpr, thresholds = roc_curve(true_labels, predictions, pos_label=1)
    
    # Compute FRR (False Rejection Rate)
    frr = 1 - tpr
    
    # Find the EER (Equal Error Rate)
    eer_index = np.argmin(np.abs(fpr - frr))
    eer = fpr[eer_index]
    eer_threshold = thresholds[eer_index]
    
    return eer, eer_threshold

# Define datasets and model predictions
datasets = [fm_generator, mg1_generator, mg2_generator, oc_generator, wm_generator]
names = ["FaceMorpher", "MIPGAN_I", "MIPGAN_II", "OpenCV", "Webmorph"]
all_results = []

# Iterate over each dataset
for dataset, name in zip(datasets, names):
#     clear_session()  # Clear session to free up memory

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(dataset, steps=dataset.samples // dataset.batch_size)
    
    # Predictions and true labels
    predictions = model.predict(dataset)
    true_labels = dataset.classes
    if predictions.ndim > 1 and predictions.shape[1] > 1:
        predictions = predictions[:, 1]

    # Calculate EER
    eer, eer_threshold = calculate_eer(true_labels, predictions)
    result = {
        "Dataset": name,
        "EER": f"{eer:.3f}",
        "Threshold": f"{eer_threshold:.3f}",
        "Test Accuracy": f"{test_accuracy:.2f}"
    }
    all_results.append(result)

# Create DataFrame
df_results = pd.DataFrame(all_results)

# Display the DataFrame
print(df_results)


9/9 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.1664 - loss: 4.9988
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 56s 6s/step - accuracy: 0.1905 - loss: 2.5083
10/10 ━━━━━━━━━━━━━━━━━━━━ 58s 6s/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 56s 6s/step - accuracy: 0.1981 - loss: 2.2891
10/10 ━━━━━━━━━━━━━━━━━━━━ 58s 6s/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.1777 - loss: 3.4952
10/10 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.2808 - loss: 3.4381
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step
       Dataset    EER Threshold Test Accuracy
0  FaceMorpher  0.500     0.001          0.19
1     MIPGAN_I  0.505     0.044          0.19
2    MIPGAN_II  0.515     0.050          0.21
3       OpenCV  0.495     0.015          0.18
4     Webmorph  0.485     0.006          0.29


Full code in a cell

In [20]:
'''
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import Xception
from tensorflow.keras.lay
ers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')
def create_data_generator(data_dir, batch_size):
    datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        zoom_range=0.1,
        brightness_range=[0.5, 1.5],
        horizontal_flip=True,
        fill_mode='nearest'
    )
    generator = datagen.flow_from_directory(
        directory=data_dir,
        target_size=(224, 224),  # Resize images to 224x224 to match the input size of the model
        batch_size=batch_size,
        class_mode='binary'  # Binary labels
    )
    return generator

# Create generators
batch_size = 32
train_generator = create_data_generator('/kaggle/input/morph-splitted/train', batch_size)
val_generator = create_data_generator('/kaggle/input/morph-splitted/val', batch_size)
test_generator = create_data_generator('/kaggle/input/morph-splitted/test', batch_size)


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, 3, activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

epochs = 10
batch_size = 1024
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)
'''


"\nimport tensorflow as tf\nfrom tensorflow.keras.preprocessing.image import ImageDataGenerator\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout\nfrom tensorflow.keras.applications import Xception\nfrom tensorflow.keras.lay\ners import Dense, GlobalAveragePooling2D\nfrom tensorflow.keras.models import Model\nimport matplotlib.pyplot as plt\nimport numpy as np\nimport pandas as pd\nfrom sklearn.metrics import roc_curve\nfrom tensorflow.keras.callbacks import EarlyStopping\nimport warnings\nwarnings.filterwarnings('ignore')\ndef create_data_generator(data_dir, batch_size):\n    datagen = ImageDataGenerator(\n        rescale=1./255,\n        rotation_range=20,\n        width_shift_range=0.2,\n        height_shift_range=0.2,\n        shear_range=0.15,\n        zoom_range=0.1,\n        brightness_range=[0.5, 1.5],\n        horizontal_flip=True,\n        fill_mode='nearest'\n    )\n    generator = datagen.flow_